# Tensorflow Basics

In [18]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm

In [19]:
tf.__version__

'2.6.0'

In [20]:
cl = tf.keras.layers.Conv2D(2, 5)
inp = tf.random.normal((1, 28, 28, 3))
out = cl(inp)
print(cl.weights[0].shape)
print(cl.weights[1].shape)

(5, 5, 3, 2)
(2,)


### Layers

In [3]:
# Multi layer perceptron
mlp = tf.keras.layers.Dense(1)
inp = tf.random.normal((1, 784))
out = mlp(inp)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2021-11-08 11:49:39.421046: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-08 11:49:39.421142: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
print(out.shape)
print("layer weights: ",mlp.weights[0].numpy().shape)
print("layer bias: ",mlp.weights[1].numpy().shape)
print("layer activation", mlp.activation)
print("layer initializers: \nweights {},\nbias {}".format(mlp.kernel_initializer, mlp.bias_initializer))
print("layer number of parameters: ", mlp.count_params())

new_weights = tf.random.uniform((784, 1), -1, 1)
new_bias = tf.random.uniform((1,), -1, 1)

mlp.set_weights([new_weights, new_bias])

assert tf.math.equal(mlp.weights[0], new_weights).numpy().any(), "Different weights"
assert tf.math.equal(mlp.weights[1], new_bias).numpy().any(), "Different bias"

(1, 1)
layer weights:  (784, 1)
layer bias:  (1,)
layer activation <function linear at 0x1658ab9d0>
layer initializers: 
weights <keras.initializers.initializers_v2.GlorotUniform object at 0x165b2ea60>,
bias <keras.initializers.initializers_v2.Zeros object at 0x165b2e970>
layer number of parameters:  785


In [5]:
# Making a model
model = tf.keras.Sequential(mlp)
out = model(inp)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (1, 1)                    785       
Total params: 785
Trainable params: 785
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# Make some dummy train data
inp_data = tf.random.uniform((1000, 784), -1, 1)
out_data = tf.random.normal((1000, 1))

In [7]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
    loss=tf.keras.losses.MSE,
)

In [8]:
model.fit(inp_data, out_data, batch_size=1, epochs=100)

Epoch 1/100
   1/1000 [..............................] - ETA: 2:28 - loss: 307.4652

2021-11-08 11:49:42.444379: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-08 11:49:42.444595: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-11-08 11:49:42.514644: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1000/1000 [==============================] - 1s 1ms/step - loss: 54.8574
Epoch 2/100
1000/1000 [==============================] - 1s 1ms/step - loss: 11.6541
Epoch 3/100
1000/1000 [==============================] - 1s 1ms/step - loss: 5.8229
Epoch 4/100
1000/1000 [==============================] - 2s 2ms/step - loss: 3.7792
Epoch 5/100
1000/1000 [==============================] - 1s 1ms/step - loss: 2.7291
Epoch 6/100
1000/1000 [==============================] - 1s 1ms/step - loss: 2.1887
Epoch 7/100
1000/1000 [==============================] - 1s 1ms/step - loss: 1.7888
Epoch 8/100
1000/1000 [==============================] - 1s 1ms/step - loss: 1.5153
Epoch 9/100
1000/1000 [==============================] - 1s 1ms/step - loss: 1.3371
Epoch 10/100
1000/1000 [==============================] - 1s 1ms/step - loss: 1.1828
Epoch 11/100
1000/1000 [==============================] - 1s 1ms/step - loss: 1.0663
Epoch 12/100
1000/1000 [==============================] - 1s 1ms/step - loss: 0.9980

In [9]:
# Make some dummy test data
inp_test_data = tf.random.uniform((1000, 784), -1, 1)
out_test_data = tf.random.normal((1000, 1))

In [10]:
# Test the model
preds = model(inp_test_data)
print(tf.reduce_mean(tf.keras.losses.mean_squared_error(out_test_data, preds)))

tf.Tensor(3.8113034, shape=(), dtype=float32)


### Gradient Tapes

In [21]:
# compute gradients dy_dx
x = tf.Variable(3.0)
with tf.GradientTape() as tape:
    y = x * x # x^2
dy_dx = tape.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [23]:
# tf Variables are automatically watched, but tensors are not
x = tf.constant(3.0)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = x * x
dy_dx = tape.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [24]:
a = tf.constant(2.0) #tf tensor
b = a + 3.0 # makes a new tensor
print(b)
print(a)

c = tf.Variable(2.0)
c.assign_add(3.0) # mutates the tensor
print(c)

tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>


In [14]:
a = tf.constant([3.0, 1.0, 2.0, 1.5])
b = a + 0.5
c = a + tf.constant([0.5, 0.5, 0.5, 0.5])
print(b)
print(c)

# Not valid (Incompatible shapes)
# c = a + tf.constant([0.5, 0.5, 0.5])


tf.Tensor([3.5 1.5 2.5 2. ], shape=(4,), dtype=float32)
tf.Tensor([3.5 1.5 2.5 2. ], shape=(4,), dtype=float32)


In [15]:
x = tf.Variable(3.0)
with tf.GradientTape(persistent=True) as tape:
    y = x * x
    z = y * y

dy_dx = tape.gradient(y, x) # dy_dx
dz_dy = tape.gradient(z, y) # dz_dy
dz_dx = tape.gradient(z, x) # dz_dy * dy_dx = 18 * 6

assert dz_dx == dz_dy * dy_dx, "Error"

print(dy_dx)
print(dz_dy)
print(dz_dx)

tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(18.0, shape=(), dtype=float32)
tf.Tensor(108.0, shape=(), dtype=float32)


In [25]:
outputs = tf.keras.layers.Dense(1)
model = tf.keras.Sequential(outputs)

In [26]:
inp_data = tf.random.uniform((1000, 784), -1, 1)
out_data = tf.random.normal(shape=(1000, 1))

pred = model(inp_data)
print(print(model.summary()))
print(model(inp_data).shape)
dataset = tf.data.Dataset.from_tensor_slices((inp_data, out_data))

for e in range(100):
    epoch_loss = list()
    for (x, y) in tqdm(dataset.batch(10)):
        with tf.GradientTape() as tape:
            # print(type(x))
            preds = model(x)
            # print(x)
            # print(preds.shape)
            loss = tf.keras.losses.mean_squared_error(y, preds)
            # print(loss.shape)
            epoch_loss.append(loss.numpy())

        dy_dw = tape.gradient(loss, model.trainable_variables)
        # print(dy_dw[0].shape)
        # print(model.variables[1].shape)
        # print(len(model.variables))#.assign_sub(0.01 * dy_dw)
        model.variables[0].assign_sub(1e-3 * dy_dw[0]) # Update the weights
        # model.variables[1].assign_sub(1e-3 * dy_dw[1]) # Update the bias
    
    print("Epoch: {}, Loss: {}".format(e, np.mean(np.array(epoch_loss))))
    

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (1000, 1)                 785       
Total params: 785
Trainable params: 785
Non-trainable params: 0
_________________________________________________________________
None
None
(1000, 1)


100%|██████████| 100/100 [00:00<00:00, 236.25it/s]


Epoch: 0, Loss: 1.5462285280227661


100%|██████████| 100/100 [00:00<00:00, 264.88it/s]


Epoch: 1, Loss: 0.8669590353965759


...

Epoch: 97, Loss: 0.3861284852027893


100%|██████████| 100/100 [00:00<00:00, 268.11it/s]


Epoch: 98, Loss: 0.3860851526260376


100%|██████████| 100/100 [00:00<00:00, 251.57it/s]

Epoch: 99, Loss: 0.3860432803630829


In [27]:
preds = model(inp_data)
train_loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(out_data, preds))
print(train_loss)

tf.Tensor(0.23978013, shape=(), dtype=float32)


In [28]:
# Make some dummy test data
inp_test_data = tf.random.uniform((1000, 784), -1, 1)
out_test_data = tf.random.normal((1000, 1))

In [29]:
preds = model(inp_test_data)
test_loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(out_test_data, preds))
print(test_loss)

tf.Tensor(4.5610785, shape=(), dtype=float32)
